In [126]:
import pandas as pd
from neo4j import GraphDatabase
df = pd.read_pickle('yummly_clean.pkl')
df = df.dropna()

In [53]:
def create_unique(session, label, value, recipeName, relationship):
    
    result = session.run("MATCH (node:" + label + ") where node.name = '" + value + "' return node")
    flag = True

    for t in result:

        flag = False
        break

    if flag:

        session.run("CREATE (n:" + label + " { name: '"+ value +"'})")
                    
    session.run("MATCH (recipe: Recipe), (c:" + label + ") WHERE recipe.name = '" + recipeName +"' AND c.name = '" + value + "' CREATE (recipe)-[r:" + relationship + "]->(c)")

In [54]:
rows = df.shape[0]
db_host = "bolt://localhost:7687"
db_username = "neo4j"
db_password = "root"
driver = GraphDatabase.driver(db_host, auth=(db_username, db_password))
with driver.session() as session:
    
    for i in range(rows):

        row = df.iloc[[i]]
        url = 'https://www.yummly.com/recipe/' + row['id'].values[0]
        recipeName = row['recipeName'].values[0].replace("'","")
        
        session.run("CREATE (recipe:Recipe {name: '"+ recipeName +"', url: '"+ url +"', rating:'"+ str(row['rating'].values[0]) +"', totalTimeInSeconds: '"+ str(row['totalTimeInSeconds'].values[0]) +"'})")
        courses = row['course'].values[0].replace('[', '').replace(']', '').split(', ')
        
        for course in courses:
            
            create_unique(session, 'Course', course, recipeName, 'IS_A')
            
        create_unique(session, 'Cuisine', row['cuisine'].values[0], recipeName, 'BELONGS_TO')
        
        for ingredient in row['clean ingredients'].values[0]:
            
            create_unique(session, 'Ingredient', ingredient, recipeName, 'CONTAINS')
        
driver.close()

KeyboardInterrupt: 

Exception ignored in: 'neobolt.bolt._io.ChunkedInputBuffer.receive'
Traceback (most recent call last):
  File "/Users/shubhamjayawant/anaconda2/envs/py36/lib/python3.6/ssl.py", line 1009, in recv_into
    return self.read(nbytes, buffer)
  File "/Users/shubhamjayawant/anaconda2/envs/py36/lib/python3.6/ssl.py", line 871, in read
    return self._sslobj.read(len, buffer)
  File "/Users/shubhamjayawant/anaconda2/envs/py36/lib/python3.6/ssl.py", line 631, in read
    v = self._sslobj.read(len, buffer)
KeyboardInterrupt: 


ServiceUnavailable: Failed to read from defunct connection Address(host='127.0.0.1', port=7687)

In [111]:
likes = ['Indian', 'American']
dislikes = ['coleslaw', 'udon', 'oregano']

In [120]:
def isAllReady(menu, limit):
    
    courses = ["Appetizers","Salads","Side Dishes","Main Dishes","Lunch and Snacks","Breakfast and Brunch","Beverages","Condiments and Sauces","Afternoon Tea","Desserts","Breads","Soups","Cocktails"]
    r = []
    
    for c in courses:
        
        if c not in menu or len(menu[c]) < limit:
            
            return False
        
    return True

In [121]:
def get_menu(session, likes, dislikes):
    
    limit = 5
    menu = {}
    final_result = []
    
    for like in likes:
        
        query = 'MATCH (rec:Recipe) -[r:BELONGS_TO]-> (c:Cuisine), (rec) -[r2:CONTAINS]-> (i:Ingredient) WHERE c.name = "' + like + '" AND NOT i.name IN ' + str(dislikes) + 'return rec.name, rec.rating'
        current_result = list(set([(t['rec.name'], t['rec.rating']) for t in session.run(query)]))
        current_result.sort(key=lambda elem: elem[1], reverse = True)
        final_result += [x[0] for x in current_result]
    
    for r in final_result:
        
        if isAllReady(menu, limit):
            
            return menu
        
        query = 'MATCH (rec:Recipe) -[r:IS_A]-> (c: Course) return c.name'
        result = session.run(query)
        
        for t in result:
            
            if t['c.name'] not in menu:
                
                menu[t['c.name']] = [r]
                break
                
            elif len(menu[t['c.name']]) < limit:
                
                menu[t['c.name']].append(r)
                break
                
    return menu

In [125]:
from py2neo import Graph
db_host = "localhost"
db_port = 7687
db_username = "neo4j"
db_password = "root"
graph = Graph(username = db_username, password = db_password, host = db_host, port = db_port)
print(get_menu(graph, likes, dislikes))

{'Appetizers': ['Indian Style Rice', 'Indian Fish Coconut Curry', 'Indian Butter Chicken', 'Indian-Spiced Mixed Nuts', 'Spicy Red Lentil and Chickpea Stew (Paulas Moroccan Lentil Stew)'], 'Salads': ['Zucchini and Tomato Curry', 'Kerala Spiced Peas', 'Mango Lassi', 'Indian Spiced Cauliflower and Potato', 'Strawberry Lassi'], 'Side Dishes': ['Curried Ground Turkey with Potatoes', 'Skinny Chicken Tikka Masala', 'Curried Sauteed Cauliflower', 'Basmati Rice', 'Keema (Indian-Style Ground Meat)'], 'Main Dishes': ['Easy Mango Lassi', 'Tandoori Chicken', 'Chana Masala', 'Middle Eastern Spicy Ground Beef with Baharat Seasoning, Mint, and Cilantro', 'Chicken Tikka Masala'], 'Lunch and Snacks': ['Indian Sweet Bread', 'Dahi Chicken Curry, Doi Murgh', 'Masala Chai (Indian Spiced Tea)', 'Goan Curried Braised Beef With Coconut Milk, Vinegar and Potatoes', 'BOMBAY ALOO'], 'Breakfast and Brunch': ['Tandoori Turkey Breast', 'Slow Cooker Lamb Rogan Josh', 'Indian Roasted Red Lentil Soup', 'Squash & Red Le